In [ ]:
! pip3 install dalex
! pip install lime

from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:
import csv
import dalex as dx
import numpy as np
import pandas as pd
import random
import sys
from itertools import combinations
from lime import lime_tabular
from pathlib import Path
from sklearn.feature_selection import SelectFpr, chi2
from sklearn.linear_model import LogisticRegression
np.random.seed(1)

def isfloat(value):
  try:
    float(value)
    return True
  except ValueError:
    return False

def get_rank_diff(list1, list2):
  avg_diff = 0
  for rd_idx in range(len(list1)):
    rank_str = list1[rd_idx]
    diff = 0
    try:
      diff = abs(list2.index(rank_str) - rd_idx)
    except ValueError:
      diff = len(list1)
    avg_diff += diff
  return (avg_diff / len(list1))

def get_hit_rate(list1, list2):
  hr = 0
  for str_hr in list1:
    if str_hr in list2:
      hr += 1
  return (hr / len(list1))

def get_random_train_comp(name_project, list_data):
  temp_list = [x for x in list_data if not name_project in x]
  return_list = []
  for i in range(2):
    temp_choice = random.choice(temp_list)
    return_list.append(temp_choice)
    temp_list.remove(temp_choice)
    
  return return_list

data_list = ['activemq-5.0.0','activemq-5.1.0','activemq-5.2.0','activemq-5.3.0','activemq-5.8.0',
             'camel-1.4.0', 'camel-2.9.0','camel-2.10.0','camel-2.11.0',
             'derby-10.2.1.6','derby-10.3.1.4','derby-10.5.1.1',
             'groovy-1_5_7','groovy-1_6_BETA_1','groovy-1_6_BETA_2',
             'hbase-0.94.0','hbase-0.95.0','hbase-0.95.2',
             'hive-0.9.0','hive-0.10.0','hive-0.12.0',
             'jruby-1.1','jruby-1.4.0','jruby-1.5.0','jruby-1.7.0.preview1',
             'lucene-2.3.0','lucene-2.9.0','lucene-3.0.0','lucene-3.1',
             'wicket-1.3.0-beta2','wicket-1.3.0-incubating-beta-1','wicket-1.5.3']

data_lists = ['wicket-1.3.0-incubating-beta-1','wicket-1.5.3']


# exp modes: give custom names in comand-line arguments for different FS algorithms
# see Apply Data Feature Selection below

f_write = open('/content/gdrive/MyDrive/out/rq5_cpdp_top_10_4.csv', 'w')
csv_writer = csv.writer(f_write)
csv_writer.writerow(['dataset', 'lime_hit_rate', 'lime_rank_diff', 'dalex_hit_rate', 'dalex_rank_diff', 'base_shape', 'target_shape', 'train1', 'train2'])

# iterate through datasets
file_cnt = 0
while file_cnt < len(data_lists):
# for data_files in data_lists:
  for ten in range(10):
    data_files = data_lists[file_cnt]
    print(data_files)
    test_df =  pd.read_csv('/content/gdrive/MyDrive/dataset/' + data_files + '.csv')
    test_df = test_df.drop('File', axis=1)
    test_x_ori = test_df.drop(['HeuBug', 'HeuBugCount', 'RealBug', 'RealBugCount'], axis=1) 
    test_real_y = test_df['RealBug']
    test_real_y = test_real_y.astype('bool')

    project_name = data_files.split('-')[0]

    train_list = get_random_train_comp(project_name, data_list)
    train = train_list[0]
    train_comp = train_list[1]

    train_df = pd.read_csv('/content/gdrive/MyDrive/dataset/' + train + '.csv')
    train_df = train_df.drop('File', axis=1)
    train_comp_df = pd.read_csv('/content/gdrive/MyDrive/dataset/' + train_comp + '.csv')
    train_comp_df = train_comp_df.drop('File', axis=1)

    #### Separate x and y for both train and test set ####
    train_x = train_df.drop(['HeuBug', 'HeuBugCount', 'RealBug', 'RealBugCount'], axis=1) 
    train_real_y = train_df['RealBug']
    train_real_y = train_real_y.astype('bool')
    feature_names = np.array(list(train_x.columns))

    train_comp_x = train_comp_df.drop(['HeuBug', 'HeuBugCount', 'RealBug', 'RealBugCount'], axis=1)
    train_comp_real_y = train_comp_df['RealBug']
    train_comp_real_y = train_comp_real_y.astype('bool')

    #### apply Spearman Corr based FS on baseline ####
    corr_mat = train_x.corr('spearman')
    corr_features = set()
    for cor_i in range(len(corr_mat.columns)):
      for cor_j in range(cor_i):
        if abs(corr_mat.iloc[cor_i, cor_j]) > 0.7:
          colname = corr_mat.columns[cor_i]
          corr_features.add(colname)
    train_x = train_x.drop(labels=corr_features, axis=1)
    test_x = test_x_ori.drop(labels=corr_features, axis=1)

    corr_mat_cmp = train_comp_x.corr('spearman')
    corr_features_cmp = set()
    for cor_i in range(len(corr_mat_cmp.columns)):
      for cor_j in range(cor_i):
        if abs(corr_mat_cmp.iloc[cor_i, cor_j]) > 0.7:
          colname = corr_mat_cmp.columns[cor_i]
          corr_features_cmp.add(colname)
    train_comp_x = train_comp_x.drop(labels=corr_features_cmp, axis=1)
    test_comp_x = test_x_ori.drop(labels=corr_features_cmp, axis=1)
    # num_of_feature = len(train_x.columns)
    num_of_feature = 10
    # print('train_x', train_x.shape)
    # print('train_comp_x:', train_comp_x.shape)  

    #### Train Classifiers (Logistic Regression) ####
    lr = LogisticRegression(random_state=0, solver='liblinear')
    lr_pred = lr.fit(train_x, train_real_y).predict(test_x)

    lr_fs = LogisticRegression(random_state=0)
    lr_fs_pred = lr_fs.fit(train_comp_x, train_comp_real_y).predict(test_comp_x)

    # if nothing predicted as buggy in this fold skip this fold
    buggy_cnt = 0
    for bc in range(len(lr_pred)):
      if lr_pred[bc] == True and lr_fs_pred[bc] == True:
        buggy_cnt += 1
    # print(buggy_cnt)
    # buggy_cnt = np.count_nonzero(lr_pred)
    if buggy_cnt == 0:
      continue

    explainer_lime = lime_tabular.LimeTabularExplainer(train_x.to_numpy(),
                                    mode='classification',
                                    feature_names=train_x.columns.tolist(),
                                    discretize_continuous=True,
                                    random_state=42)
    explainer_lime_fs = lime_tabular.LimeTabularExplainer(train_comp_x.to_numpy(),
                                    mode='classification',
                                    feature_names=train_comp_x.columns.tolist(),
                                    discretize_continuous=True,
                                    random_state=42)
    exp_dalex = dx.Explainer(lr, train_x, train_real_y, verbose=False)
    exp_dalex_fs = dx.Explainer(lr_fs, train_comp_x, train_comp_real_y, verbose=False)

    # 'lime_hit_rate', 'lime_rank_diff', 'dalex_hit_rate', 'dalex_rank_diff'
    avg_hit_rate_lime = 0
    avg_rank_diff_lime = 0
    avg_hit_rate_dx = 0
    avg_rank_diff_dx = 0
    cnt = 0
    for i in range(len(lr_pred)):
      if lr_pred[i] == True and lr_fs_pred[i] == True:
        #### Dalex (BreakDown's new package) ####
        # Create explainer for original model
        breakdown = exp_dalex.predict_parts(test_x.iloc[i], type='break_down', label=str(i))
        # interactions = exp_dalex.predict_parts(test_x.iloc[i], type='break_down_interactions', label=str(i)+'+')
        # breakdown.plot(interactions)
        result_df = breakdown.result
        # print(result_df.iloc[1:].head(top_k_features).to_string)

        # Create Explainer for feature selected model
        breakdown_fs = exp_dalex_fs.predict_parts(test_comp_x.iloc[i], type='break_down', label=str(i))
        # interactions_fs = exp_dalex_fs.predict_parts(test_comp_x.iloc[i], type='break_down_interactions', label=str(i)+'+')
        # breakdown_fs.plot(interactions_fs)
        result_df_fs = breakdown_fs.result
        # print(result_df_fs.iloc[1:].head(top_k_features).to_string)

        #### LIME ####
        exp_lime = explainer_lime.explain_instance(test_x.to_numpy()[i], lr.predict_proba, num_features=num_of_feature)
        lime_result = pd.DataFrame(exp_lime.as_list(), columns=['features','score'])
        
        exp_lime_fs = explainer_lime_fs.explain_instance(test_comp_x.to_numpy()[i], lr_fs.predict_proba, num_features=num_of_feature)
        lime_result_fs = pd.DataFrame(exp_lime_fs.as_list(), columns=['features','score'])

        
        # get the ranking list of original model (dalex)
        rank_list = result_df['variable_name'].tolist()
        del rank_list[0]
        del rank_list[len(rank_list) - 1]

        # get the ranking list of feature selected model (dalex)
        rank_list_fs = result_df_fs['variable_name'].tolist()
        del rank_list_fs[0]
        del rank_list_fs[len(rank_list_fs) - 1]

        # get the ranking list of original model (lime)
        rank_list_lime = lime_result['features'].tolist()
        for k in range(len(rank_list_lime)):
          split = rank_list_lime[k].split()
          if isfloat(split[0]):
            rank_list_lime[k] = split[2]
          else:
            rank_list_lime[k]= split[0]
        # print(rank_list_lime)

        # get the ranking list of feature selected model(lime)
        rank_list_fs_lime = lime_result_fs['features'].tolist()
        for k in range(len(rank_list_fs_lime)):
          split = rank_list_fs_lime[k].split()
          if isfloat(split[0]):
            rank_list_fs_lime[k] = split[2]
          else:
            rank_list_fs_lime[k]= split[0]
        # print(rank_list_fs_lime)

        # trunc list for BreakDown
        rank_list = rank_list[0:num_of_feature]
        rank_list_fs = rank_list_fs[0:num_of_feature]

        # get ranking difference for lime and dalex
        avg_rank_diff_lime += get_rank_diff(rank_list_lime, rank_list_fs_lime)
        avg_rank_diff_dx += get_rank_diff(rank_list, rank_list_fs)

        # get hit rate for lime and dalex
        avg_hit_rate_lime += get_hit_rate(rank_list_lime, rank_list_fs_lime)
        avg_hit_rate_dx += get_hit_rate(rank_list, rank_list_fs)
        # print(cnt, '/', buggy_cnt)
        cnt += 1
    avg_hit_rate_lime /= buggy_cnt
    avg_rank_diff_lime /= buggy_cnt
    avg_hit_rate_dx /= buggy_cnt
    avg_rank_diff_dx /= buggy_cnt
    row = [data_files, avg_hit_rate_lime, avg_rank_diff_lime, avg_hit_rate_dx, avg_rank_diff_dx, train_x.shape, train_comp_x.shape, train, train_comp]
    csv_writer.writerow(row)
    print(row)
    f_write.flush()
  file_cnt += 1
f_write.close()
print('Done!')